In [1]:
!pip install datasets
!pip install transformers
!pip install rouge
!pip install spacy

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [9]:
from datasets import load_dataset
import re
# from nltk.tokenize import sent_tokenize
# import nltk
# import numpy as np
import tqdm
import spacy
# nltk.download('punkt')

In [10]:
from datasets import load_dataset
dataset = load_dataset('multi_news', split='train')

In [11]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)

In [12]:
nlp = spacy.load("en_core_web_sm")

In [13]:
def score_sentence(document):
    text = document['document']
    doc = nlp(text)
    rouge_scores = []
    entity_counts = []
    
    
    for s in doc.sents:
        target = text[s.start_char:s.end_char]
        rest_doc = text[:s.start_char] + text[s.end_char:]
#         print(target)
        try:
            score = scorer.score(target,
                      rest_doc)['rouge1'][2]
            rouge_scores.append(score)
        except:
            rouge_scores.append(0)
        

        entity_counts.append(len(nlp(str(s)).ents))   
            
    document['rouge_scores'] = rouge_scores
    document['entity_counts'] = entity_counts
    
#     rank = (-np.array(scores)).argsort()
#     threshold = int(len(rank)*0.8)
#     rank = rank[:threshold]
#     rank.sort()
#     filtered_doc = ' '.join([splited_document[i] for i in rank])
#     doc['document'] = filtered_doc
    return document

In [17]:
small_dataset = dataset.select(list(range(500)))

small_dataset = small_dataset.add_column("rouge_scores", [[0]] * len(small_dataset))
small_dataset = small_dataset.add_column("entity_counts", [[0]] * len(small_dataset))

In [18]:
small_dataset

Dataset({
    features: ['document', 'summary', 'rouge_scores', 'entity_counts'],
    num_rows: 500
})

In [19]:
import time 
start  = time.time()
d = small_dataset.map(score_sentence, num_proc = 48)
end = time.time()
print (end-start)

#1:   0%|          | 0/11 [00:00<?, ?ex/s]

#0:   0%|          | 0/11 [00:00<?, ?ex/s]

#9:   0%|          | 0/11 [00:00<?, ?ex/s]

#2:   0%|          | 0/11 [00:00<?, ?ex/s]

#12:   0%|          | 0/11 [00:00<?, ?ex/s]

...

#41:   0%|          | 0/10 [00:00<?, ?ex/s]

#39:   0%|          | 0/10 [00:00<?, ?ex/s]

#42:   0%|          | 0/10 [00:00<?, ?ex/s]

#44:   0%|          | 0/10 [00:00<?, ?ex/s]

1016.7363572120667


In [21]:
d.save_to_disk("/scratch/kd1860/DSGA_1006_capstone/dataset/scored_dataset_test500")

In [22]:
from datasets import load_from_disk
reloaded_dataset = load_from_disk("/scratch/kd1860/DSGA_1006_capstone/dataset/scored_dataset_test500")

In [25]:
reloaded_dataset

Dataset({
    features: ['document', 'summary', 'rouge_scores', 'entity_counts'],
    num_rows: 500
})